## LLM as solver for Traveling salesman problem

In [11]:
from dataclasses import dataclass, field
from adalflow.core.base_data_class import DataClass
from typing import List, Dict
from adalflow.components.output_parsers.dataclass_parser import DataClassParser

@dataclass
class TSPSolution(DataClass):
    __doc__ = """You are given two inputs.\n
    
    1. the distances betwen n cities represented as a dictionary.
    Each key in the dictionary represents a city. The value is a list of distance
    of the city in key to all the other cities.
    
    2. starting node, aka starting city
    
    You need to find out the shortest path to travel to all the cities from starting node and return
    back to starting node. This is famously called as Travelling Sales Man problem, a varian of
    hamiltonian circuit in a graph.
    """
     
    distance_matrix: Dict = field(
        metadata={"desc": "A dictionary, where each key is a city and the value is the list of distance to all other cities"}
    )
    
    start_node: int = field(
        metadata={"desc": "starting city. The city to start and end the path."}
    )
    
    shortest_path: List = field(
        metadata={"desc": "List of nodes to be traversed in order"}
    )
    score: int = field(
        metadata={"desc":"total score in traversing from start node covering all nodes and back to start node." }
    )

TSPSolution.set_input_fields(["distance_matrix","start_node"])
TSPSolution.set_output_fields(["shortest_path", "score"])


In [12]:
from adalflow.components.output_parsers.dataclass_parser import DataClassParser

solution_parser = DataClassParser(data_class=TSPSolution)
print(solution_parser.get_input_format_str())


{
    "distance_matrix": "A dictionary, where each key is a city and the value is the list of distance to all other cities (Dict) (required)",
    "start_node": "starting city. The city to start and end the path. (int) (required)"
}


In [13]:
import numpy as np

def nnsolve(distance_matrix, start):
    """A brute force algorithm from chatgpt"""
    n = len(distance_matrix)
    cities = list(range(n))
    min_path = None
    min_cost = float('inf')

    def backtrack(current_city, visited, path, cost):
        nonlocal min_path, min_cost
        if len(visited) == n:
            total_cost = cost + distance_matrix[current_city][start]
            if total_cost < min_cost:
                min_cost = total_cost
                min_path = path + [start]
            return

        for next_city in cities:
            if next_city not in visited:
                visited.add(next_city)
                backtrack(next_city, visited, path + [next_city], cost + distance_matrix[current_city][next_city])
                visited.remove(next_city)

    visited = {start}
    backtrack(start, visited, [start], 0)

    return min_path, min_cost

class GenerateTSP:
    """
    A helper class to generate tiny travelling salesman problems
    """
    def __init__(self, n_cities: int = 5, max_distance: int=35, solver=nnsolve) -> None:
        """
        Args:
            n: number of cities
            max_distance: maximum distance between two cities
        """
        self.n_cities = n_cities
        self.distance_matrices = []
        self.max_distance = max_distance
        self.solver = nnsolve
        
        
    def __generate_dist_matrices(self,n: int=1):
        """Generate n random distance matrix"""
        d_matrices = np.random.randint(low=1, high=self.max_distance,\
                    size=(n, self.n_cities, self.n_cities))
        for i in range(n):
            np.fill_diagonal(d_matrices[i], 0)
        self.distance_matrices = d_matrices
    
    def generate_tsp(self, n_shots = 3):
        """"""
        self.__generate_dist_matrices(n=n_shots)
        tsp_problems = []
        
        for dist_matrix in self.distance_matrices:
            
            start_node =  int(np.random.randint(0,self.n_cities,dtype=int))
            shortest_path,score = self.solver(distance_matrix=dist_matrix, start=start_node)
            solution = TSPSolution(
                 distance_matrix = {int(idx): dist_matrix[idx].tolist() for idx in range(self.n_cities)}
                ,start_node = int(start_node)
                ,shortest_path =  [int(x) for x in shortest_path]
                ,score = int(score)
            )
            tsp_problems.append(solution)
        return tsp_problems

In [14]:
gen_tsp = GenerateTSP(n_cities=10)
tsps = gen_tsp.generate_tsp(n_shots=1)
tsps

[TSPSolution(distance_matrix={0: [0, 6, 25, 15, 14, 22, 19, 30, 17, 12], 1: [26, 0, 18, 26, 11, 3, 1, 22, 30, 28], 2: [15, 6, 0, 19, 27, 11, 3, 30, 21, 29], 3: [12, 25, 29, 0, 18, 9, 6, 32, 4, 19], 4: [26, 19, 25, 20, 0, 31, 28, 6, 8, 4], 5: [26, 16, 18, 8, 17, 0, 31, 31, 17, 13], 6: [22, 19, 25, 34, 7, 12, 0, 9, 7, 11], 7: [22, 29, 23, 27, 16, 16, 25, 0, 10, 32], 8: [17, 28, 20, 12, 33, 33, 2, 33, 0, 8], 9: [28, 34, 26, 25, 21, 18, 9, 25, 34, 0]}, start_node=1, shortest_path=[1, 5, 3, 8, 9, 6, 4, 7, 2, 0, 1], score=89)]

In [15]:
from adalflow.core import Prompt

prompt_template=r"""
<SYS>You are great math assistant.Carefully read the user query. Divide it into steps if needed to solve. 
Solve it accurately.
{# task_desc #}
{% if task_desc_str %}
{{task_desc_str}}
{% endif %}
The input will be provided in
the following format.
{# input_format_str #}
{% if input_format_str %}
{{input_format_str}}
{% endif %}
{# input examples #}
{% if input_examples %}
Here are some example input.
{{input_examples}}
{% endif%}
{# output format #}
{% if output_format_str %}
{{output_format_str}}
{% endif %}
{% if output_examples %}
Here are some examples for how the output
should be generated
    {{output_examples}}
{% endif%}
</SYS>
Find the shortest path from the travelling salesman problem {{tsp_problem}} 
You:
"""

prompt_kwargs = {
    
    "task_desc_str":     solution_parser.get_task_desc_str()
   ,"input_format_str":  solution_parser.get_input_format_str()
   ,"input_examples":    solution_parser.get_examples_str(examples=tsps, exclude=["shortest_path","score"])
   ,"output_format_str": solution_parser.get_output_format_str()
   ,"output_examples" :  solution_parser.get_examples_str(examples=tsps,exclude=["distance_matrix","start_node"])
}
print(solution_parser.get_examples_str(examples=tsps, exclude=["shortest_path","score"]))


{
    "distance_matrix": {
        "0": [
            0,
            6,
            25,
            15,
            14,
            22,
            19,
            30,
            17,
            12
        ],
        "1": [
            26,
            0,
            18,
            26,
            11,
            3,
            1,
            22,
            30,
            28
        ],
        "2": [
            15,
            6,
            0,
            19,
            27,
            11,
            3,
            30,
            21,
            29
        ],
        "3": [
            12,
            25,
            29,
            0,
            18,
            9,
            6,
            32,
            4,
            19
        ],
        "4": [
            26,
            19,
            25,
            20,
            0,
            31,
            28,
            6,
            8,
            4
        ],
        "5": [
            26,
            16,
            18,


In [18]:
from adalflow.components.model_client.ollama_client import OllamaClient
from adalflow.core.generator import Generator
from adalflow.core.component import Component
from adalflow.core.string_parser import JsonParser



class TSPSolver(Component):
    """
    Component to solve a Travelling Salesman Problem
    """
    def __init__(self, prompt_template, prompt_kwargs):
        super().__init__()
        host = "127.0.0.1:11434"
        self.generator = Generator(
            model_client = OllamaClient(host=host)
           ,model_kwargs = {"model":"phi3:latest"}
           ,template = prompt_template
           ,prompt_kwargs = prompt_kwargs
           ,name ="Funny Generator" 
        )
        
        self.output_parser = JsonParser()
    
    def call(self, tsp_problem):
        input_problem = {"tsp_problem": tsp_problem}
        result = self.generator(input_problem)
        solution = self.output_parser(result.data)
        
        return solution

In [19]:
a_tsp_problem = gen_tsp.generate_tsp(n_shots=1)
tsp_problem = solution_parser.get_examples_str(examples=a_tsp_problem, exclude=["shortest_path","score"])

tsp_solver = TSPSolver(prompt_template, prompt_kwargs)
result = tsp_solver(tsp_problem)

Error calling the model: [Errno 111] Connection refused
Error calling the model: [Errno 111] Connection refused


AttributeError: 'NoneType' object has no attribute 'strip'

In [ ]:
result

## Generate data and test

In [ ]:
from tqdm import tqdm

tsp_problems = gen_tsp.generate_tsp(n_shots=5)

paths_gt = []
scores_gt = []

paths_predicted  = []
scores_predicted = []

for i in tqdm(range(len(tsp_problems))):

    paths_gt.append(tsp_problems[i].shortest_path)
    scores_gt.append(tsp_problems[i].score)

    tsp_problem = solution_parser.get_examples_str(examples=[tsp_problems[i]], exclude=["shortest_path","score"])
    tsp_solver = TSPSolver(prompt_template, prompt_kwargs)
    result = tsp_solver(tsp_problem)

    paths_predicted.append(result['shortest_path'])
    scores_predicted.append(result['score'])


In [ ]:
import numpy as np

np.mean(np.abs(np.asarray(scores_gt) - np.asarray(scores_predicted)))